# Reproducting Graph Neural Collaborative Filtering

[Link to original paper](https://arxiv.org/abs/1905.08108)

Authors: 
 - @MathiasMeuleman
 - @SytzeAndr
 - @sinuochen

In [61]:
! pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [62]:
import numpy as np
import csv
import scipy.sparse as sp
import torch
import torch_geometric

from pathlib import Path

In [63]:
from google.colab import drive

# Mount drive for permanent file storage
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Utilities

In [64]:
def split_mtx(X, n_folds=200):
  """
  Split a matrix/Tensor into n parts.
  Useful for processing large matrices in batches
  """
  X_folds = []
  fold_len = X.shape[0]//n_folds
  for i in range(n_folds):
    start = i * fold_len
    if i == n_folds -1:
      end = X.shape[0]
    else:
      end = (i + 1) * fold_len
    X_folds.append(X[start:end])
  return X_folds

def to_sparse_tensor(X):
  """
  Convert a sparse numpy object to a sparse pytorch tensor.
  Note that the tensor does not yet live on the GPU
  """
  coo = X.tocoo().astype(np.float32)
  i = torch.LongTensor(np.mat((coo.row, coo.col)))
  v = torch.FloatTensor(coo.data)
  return torch.sparse.FloatTensor(i, v, coo.shape)


# Data Loading

The `DataLoader` class mainly takes care of loading the data. In addition, it has functions to compute and store the adjacency matrix and can be used for easy batch sampling.

In [65]:
path = Path('./drive/My Drive/NGCF_data/')
train_file = path/'train.txt'
test_file = path/'test.txt'

class DataLoader:
  def __init__(self, file, batch_size):
    self.file = file
    self.batch_size = batch_size
    self.n_users, self.n_items, self.n_data = 0, 0, 0
    self.users = []
    self.pos_items = {}
    self.neg_items = {}
    self.load()

  def load(self):
    with open(self.file) as f:
      for l in f.readlines():#每个user买了什么item（in train.txt）
        if len(l) == 0: break
        l = l.strip('\n').split(' ')

        uid = int(l[0])
        try:
          items = [int(i) for i in l[1:]]
        except Exception:
          continue
        self.users.append(uid)
        self.n_items = max(self.n_items, max(items))#目前取得的最大的 item id
        self.n_users = max(self.n_users, uid)   #目前取得的最大的 user id
        self.n_data += len(items)          #n_data存所有user"买"这个行为的次数
        self.pos_items[uid] = items         #记录某个user买了的items，pos_items 是一个二维数组
    self.n_users += 1
    self.n_items += 1
    
    # R is the Rating matrix in Dict Of Keys form, either 1. or 0. for each (user, item) pair
    self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
    
    for u in self.users:
      for i in self.pos_items[u]:
        self.R[u, i] = 1.
    #self.R成为一个shape(29858，40982)的matrix，如果一个user-item pair 存在，为1，不存在则为0
    #print("self.R",self.R.get_shape())

  def compute_norm_adj_matrix(self, adj):
    #axis 0 即返回一行数（每列的和），axis 1 即返回一列（每行的和）   
    rowsum = np.array(adj.sum(1))
    # inverted and set to 0 if no connections
    # rowsum的each element^(-1), 然后让其变为一行
    d_inv = np.power(rowsum, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    # sparse diagonal matrix with the normalizing factors in the diagonal
    # Construct a sparse matrix from diagonals.
    d_mat_inv = sp.diags(d_inv)
    # dot product resulting in a row-normalised version of the input matrix
    norm_adj = d_mat_inv.dot(adj)
    return norm_adj.tocoo()
  
  def set_adj_matrix(self):
    try:
      self.adj_matrix = sp.load_npz(path/'s_adj_mat.npz')
      print('Loaded existing adj_matrix')
    except Exception:
      print('No exisiting adj_matrix found')
      adj = self.compute_adj_matrix()
      sp.save_npz(path/'s_adj_mat.npz', adj.tocsr())
      self.adj_matrix = adj

  def compute_adj_matrix(self):
    # A is the Adjecency matrix in Dict Of Keys form, used when computing the Laplacian norm
    A = sp.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32).tolil()
    A[:self.n_users, self.n_users:] = self.R.tolil()  ##返回sparse matrix 的lil_matrix形式Convert this matrix to List of Lists format.
    A[self.n_users:, :self.n_users] = self.R.tolil().T
    A = A.todok() #返回sparse matrix 的 dok_matrix形式

    # norm_adj = self.compute_norm_adj_matrix(A + sp.eye(A.shape[0]))
    mean_adj = self.compute_norm_adj_matrix(A)
    # L is the Laplacian used for normalizing message construction
    # ngcf_adj = mean_adj + sp.eye(mean_adj.shape[0])
    self.adj_matrix = mean_adj + sp.eye(mean_adj.shape[0])#sp.eye(num of rows in the matrix) Returns a sparse (m x n) matrix where the kth diagonal is all ones and everything else is zeros
    return mean_adj + sp.eye(mean_adj.shape[0])
 

  """
    For each observed user-item interaction, we treat it as a positive
  instance, and then conduct the negative sampling strategy to pair
  it with one negative item that the user did not consume before.
  """
  def sample_pos(self, u, amount):
    # Sample a batch of <amount> positive items for user u
    high = len(self.pos_items[u])
    pos_sample = []
    while len(pos_sample) < amount:
      id = np.random.randint(low=0, high=high, size=1)[0]
      item = self.pos_items[u][id]
      if item not in pos_sample:
        pos_sample.append(item)
    return pos_sample

  def sample_neg(self, u, amount):
    # Sample a batch of <amount> negative items for user u
    high = self.n_items
    neg_sample = []
    while len(neg_sample) < amount:
      item = np.random.randint(low=0, high=high, size=1)[0]
      if item not in self.pos_items[u] and item not in neg_sample:
        neg_sample.append(item)
    return neg_sample

  def sample(self):
    # Sample a batch of batch_size users, each with a positive and negative item
    users = np.random.choice(self.users, size=self.batch_size) #随机从0-self.users中选出self.batch_size个整数，变成np array之后传给users
    pos_sample, neg_sample = [], []
    for u in users:
      pos_sample += self.sample_pos(u, 1)
      neg_sample += self.sample_neg(u, 1)
    
    # print("users", users.shape)      #1024
    # print("pos_sample", len(pos_sample)) #1024
    # print("neg_sample", len(neg_sample)) #1024
    return users, pos_sample, neg_sample

train_data = DataLoader(train_file, batch_size=1024)
train_data.set_adj_matrix()
test_data = DataLoader(test_file, batch_size=1024)

Loaded existing adj_matrix


# Model definition

The NGCF class defines the network for the NGCF framework. The forward function performs message construction and message aggregation, followed by computing the pairwise BPR loss on the predictions.

In [66]:
from torch.nn import init, LeakyReLU, Linear, Module, ModuleList, Parameter
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

# path to save model to
models_path = path/'models/2layer'

class NGCF(Module):
  def __init__(self, n_users, n_items, embed_size, n_layers, adj_matrix):
    super().__init__()
    self.n_users = n_users
    self.n_items = n_items
    self.embed_size = embed_size
    self.n_layers = n_layers
    self.adj_matrix = adj_matrix

    # The (user/item)_embeddings are the initial embedding matrix E
    self.user_embeddings = Parameter(torch.rand(n_users, embed_size))
    self.item_embeddings = Parameter(torch.rand(n_items, embed_size))
    print("\n self.user_embeddings", self.user_embeddings.shape)# self.user_embeddings torch.Size([29858, 64])
    print("\n self.item_embeddings", self.item_embeddings.shape)# self.item_embeddings torch.Size([40981, 64])

    # The (user/item)_embeddings_final are the final concatenated embeddings [E_1..E_L]
    # Stored for easy tracking of final embeddings throughout optimization and eval
    self.user_embeddings_final = Parameter(torch.zeros((n_users, embed_size * (n_layers + 1))))#充满zero
    self.item_embeddings_final = Parameter(torch.zeros((n_items, embed_size * (n_layers + 1))))
    print("\n self.user_embeddings_final", self.user_embeddings_final.shape)# self.user_embeddings_final torch.Size([29858, 192])
    print("\n self.item_embeddings_final", self.item_embeddings_final.shape)# self.item_embeddings_final torch.Size([40981, 192])

    # The linear transformations for each layer
    self.W1 = ModuleList([Linear(self.embed_size, self.embed_size) for _ in range(0, self.n_layers)]) # y=b+W1^T * X  
    self.W2 = ModuleList([Linear(self.embed_size, self.embed_size) for _ in range(0, self.n_layers)])
    print("\n self.W1", self.W1)
    print("\n self.W2", self.W2)
    """
          self.W1 ModuleList((0-1): 2 x Linear(in_features=64, out_features=64, bias=True))
          self.W2 ModuleList((0-1): 2 x Linear(in_features=64, out_features=64, bias=True))
    """
    
    self.act = LeakyReLU()
    
    # Initialize each of the trainable weights with the Xavier initializer
    self.init_weights()

  

  def init_weights(self):#initialize W1 W2
    
    for name, parameter in self.named_parameters():
      
      if ('bias' not in name):
        init.xavier_uniform_(parameter)

  def compute_loss(self, batch_user_emb, batch_pos_emb, batch_neg_emb):
    pos_y = torch.mul(batch_user_emb, batch_pos_emb).sum(dim=1)
    neg_y = torch.mul(batch_user_emb, batch_neg_emb).sum(dim=1)
    # Unregularized loss
    bpr_loss = -(torch.log(torch.sigmoid(pos_y - neg_y))).mean()
    return bpr_loss

  def message(self, x_i, x_j, norm):
    # Construct message
    message = self.lin1(x_j)
    # Only add the second term if it is not a self loop
    if x_i.data_ptr() != x_j.data_ptr():
      message += self.lin2(torch.mul(x_i, x_j))
    return message

  def update(self, aggr_out):
    # Return the LeakyReLU result
    return self.act(aggr_out)

  def forward(self, u, i, j):
    #print("self.adj_matrix", self.adj_matrix)
    adj_splits = split_mtx(self.adj_matrix)
    #print("self.adj_splits", adj_splits)
    #the connected user-item pair
    embeddings = torch.cat((self.user_embeddings, self.item_embeddings))#E=[e_u1,...,e_uN, e_i1...,e_iM ]
    final_embeddings = [embeddings]

    for l in range(self.n_layers):
      embedding_parts = []
      for part in adj_splits:
        embedding_parts.append(torch.sparse.mm(to_sparse_tensor(part).to(device), embeddings))

      # Message construction
      t1_embeddings = torch.cat(embedding_parts, 0)
      t1 = self.W1[l](t1_embeddings)
      t2_embeddings = embeddings.mul(t1_embeddings)
      t2 = self.W2[l](t2_embeddings)

      # Message aggregation
      embeddings = self.act(t1 + t2)
      normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
      final_embeddings.append(normalized_embeddings)

    # Make sure to update the (user/item)_embeddings(_final)
    final_embeddings = torch.cat(final_embeddings, 1)
    final_u_embeddings, final_i_embeddings = final_embeddings.split((self.n_users, self.n_items), 0)
    self.user_embeddings_final = Parameter(final_u_embeddings)
    self.item_embeddings_final = Parameter(final_i_embeddings)

    batch_user_emb = final_u_embeddings[u]
    batch_pos_emb = final_i_embeddings[i]
    batch_neg_emb = final_i_embeddings[j]

    return self.compute_loss(batch_user_emb, batch_pos_emb, batch_neg_emb)

Define the model and optimizer. Includes a utility function to restore the state of the model and optimizer. Handy when training for a long time and finding that Google suddenly restricted yout GPU time.

In [67]:
# optional: restore previously trained model
def restore_model(path, model, optimizer):
  model.load_state_dict(torch.load(path/'model1.pt'))
  optimizer.load_state_dict(torch.load(path/'optimizer.pt'))
  print('Restored previous model')
  return model, optimizer

# norm_adj, mean_adj, ngcf_adj = train_data.compute_adj_matrix()
device = torch.device('cuda')
model = NGCF(n_users=train_data.n_users, n_items=train_data.n_items, embed_size=64, n_layers=2, adj_matrix=train_data.adj_matrix).to(device)
# XXX = model.named_parameters
# print(XXX)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005) #We adopt mini-batch Adam [17] to optimize the prediction model and update the model parameters.
print((list(model.parameters())))
# Restore a previous model
model, optimizer = restore_model(models_path, model, optimizer)


 self.user_embeddings torch.Size([29858, 64])

 self.item_embeddings torch.Size([40981, 64])

 self.user_embeddings_final torch.Size([29858, 192])

 self.item_embeddings_final torch.Size([40981, 192])

 self.W1 ModuleList(
  (0-1): 2 x Linear(in_features=64, out_features=64, bias=True)
)

 self.W2 ModuleList(
  (0-1): 2 x Linear(in_features=64, out_features=64, bias=True)
)
[Parameter containing:
tensor([[-3.8381e-03, -5.4686e-03, -1.0556e-02,  ...,  2.1115e-03,
         -4.6655e-03, -1.2026e-02],
        [ 8.4868e-05, -8.1239e-04,  3.8315e-03,  ...,  1.3106e-03,
         -2.8515e-03,  1.1117e-02],
        [ 2.7791e-03, -6.5072e-03,  1.3441e-02,  ..., -9.4073e-03,
          8.0124e-03, -1.5341e-03],
        ...,
        [-3.4966e-03,  8.1531e-03, -9.3430e-03,  ...,  3.6964e-03,
          1.2547e-02, -3.1084e-03],
        [-8.0706e-04, -1.2329e-02, -1.1567e-02,  ...,  5.3000e-03,
          2.7843e-04,  9.8581e-03],
        [ 1.2708e-02,  4.2715e-04,  1.1262e-02,  ...,  1.0140e-02,
    

# Optimization and evaluation

Here, training and evaluation is peformed.


In [83]:
from time import time

n_epochs = 10

model.train()
n_batch = train_data.n_users // train_data.batch_size + 1
# print(train_data.n_data)
def compute_ndcg(top_items, test_items, test_indices, k):
  ratings = (test_items * top_items).gather(1, test_indices)
  norm = torch.from_numpy(np.log2(np.arange(2, k+2))).float().to(device)
  dcg = (ratings / norm).sum(1)
  dcg_max = (torch.sort(ratings, dim=1, descending=True)[0] / norm).sum(1)
  ndcg = dcg / dcg_max
  ndcg[torch.isnan(ndcg)] = 0
  return ndcg

def evaluate(user_embeddings, item_embeddings, k):
  user_parts = split_mtx(user_embeddings)
  train_parts = split_mtx(train_data.R)
  test_parts = split_mtx(test_data.R)

  recall_parts, ndcg_parts = [], []

  for user_part, train_part, test_part in zip(user_parts, train_parts, test_parts):

    # Get the prediction scores for the users and items as a cuda float
    non_train_items = torch.from_numpy(1 - (train_part.todense())).float().to(device)
    predictions = torch.mm(user_part, item_embeddings.t()) * non_train_items
    # Get the k highest scores, scatter them as a float tensor in the GPU
    _, test_indices = torch.topk(predictions, dim=1, k=k)
    top_items = torch.zeros_like(predictions).float()
    #print(test_indices)
    top_items.scatter_(dim=1, index=test_indices, src=torch.full(test_indices.shape, 1.0).to(device))
  
    test_items = torch.from_numpy(test_part.todense()).float().to(device)
    TP = (test_items * top_items).sum(1)
    recall = TP / test_items.sum(1)
    recall[torch.isnan(recall)] = 1
    ndcg = compute_ndcg(top_items, test_items, test_indices, k)
  
    recall_parts.append(recall)
    ndcg_parts.append(ndcg)

  mean_recall = torch.cat(recall_parts).mean()
  mean_ndcg = torch.cat(ndcg_parts).mean()
  print('Recall:\t' + str(mean_recall.item()))
  print('NDCG\t:' + str(mean_ndcg.item()))

def save_state(model, optimizer, epoch):
  torch.save(model.state_dict(), models_path/'model1.pt')
  torch.save(optimizer.state_dict(), models_path/'optimizer.pt')
  torch.save(torch.IntTensor(epoch), models_path/'epoch.pt')

def train(model, data, t):
  total_loss = 0
  start = time()
  timings = []
  for b in range(n_batch):
    
    batch_start = time()
    u, i, j = data.sample() #u是一个batch中的1024个users id， i是1024个positive sample, j是1024个negative sample
    u = torch.from_numpy(u).long().to(device)
    i = torch.LongTensor(i).to(device)
    j = torch.LongTensor(j).to(device)
    # print("\n u", u)
    # print("\n i", i)
    # print("\n j", j)
    

    optimizer.zero_grad()
    loss = model(u, i, j) #forward函数执行
    #print("loss", loss.item())
    loss.backward() #对loss中的每一个变量求偏导
    optimizer.step() #更新参数 W1(8*8) b1(8*8), W2(8*8) b2(8*8) 3层layers
    total_loss += loss.item() #把除了loss.backward()之外的对loss的调用都改成loss.item(), 否则loss会一直迭代
    timings.append(time()-batch_start) #计时

  avg_batch = np.average(timings)
  print('Finished epoch ' + str(t+1) + ' in\t' + str(time()-start) + ' sec')
  print('Total BPR loss:\t\t' + str(total_loss))
  print('Average batch time:\t' + str(avg_batch))

for t in range(n_epochs):
  print('Starting epoch: ' + str(t+1))
  train(model, train_data, t)
  save_state(model, optimizer, t)
  if (t+1) % 5 == 0:
    model.eval()
    evaluate(model.user_embeddings_final, model.item_embeddings_final, k=10)
    model.train()
  print('\n============\n')

Starting epoch: 1
Finished epoch 1 in	54.819281578063965 sec
Total BPR loss:		1.5745480693876743
Average batch time:	1.827303139368693


Starting epoch: 2
Finished epoch 2 in	54.735039710998535 sec
Total BPR loss:		1.5858487114310265
Average batch time:	1.8244951963424683


Starting epoch: 3
Finished epoch 3 in	54.73480987548828 sec
Total BPR loss:		1.5431170463562012
Average batch time:	1.8244882424672444


Starting epoch: 4
Finished epoch 4 in	54.473047494888306 sec
Total BPR loss:		1.6059352047741413
Average batch time:	1.8157624165217081


Starting epoch: 5
Finished epoch 5 in	54.66131615638733 sec
Total BPR loss:		1.5387996658682823
Average batch time:	1.822038507461548
Recall:	0.06492892652750015
NDCG	:0.18594184517860413


Starting epoch: 6
Finished epoch 6 in	55.036553144454956 sec
Total BPR loss:		1.5713114701211452
Average batch time:	1.8345462083816528


Starting epoch: 7
Finished epoch 7 in	54.921457290649414 sec
Total BPR loss:		1.4578590467572212
Average batch time:	1.830